In [ ]:
import sys
import os
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(sys.path[0]))))

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'True'
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image,ImageDraw
import easyocr
import numpy as np
import json
import re
import torch
from tqdm import tqdm
from scripts.train.utils import smart_tokenizer_and_embedding_resize,Html2BboxTree,move_to_device,BboxTree2Html,add_special_tokens
from vars import *
from my_dataset import UICoderDataset,UICoderCollater
from transformers import AutoProcessor, Pix2StructForConditionalGeneration,AddedToken
from utils import Html2BboxTree, BboxTree2StyleList, BboxTree2Html
from datasets import Dataset

torch.manual_seed(SEED)

device = 'cuda:0'
model_path = "/data02/users/lz/code/UICoder/checkpoints/stage0/l1024_p1024_vu_3m*3/checkpoint-4000"
# data_path = '/data02/users/lz/code/UICoder/datasets/WebSight-format-parquet'
data_path = '/data02/users/lz/code/UICoder/datasets/WebSight-format-parquet/arrow'
# data_path = '/data02/starmage/datasets/cc/arrows_8-14_processed/'
output_dir = '/data02/users/lz/code/UICoder/test_result'

In [ ]:
processor = AutoProcessor.from_pretrained(processor_name_or_path)
model = Pix2StructForConditionalGeneration.from_pretrained(model_path,is_encoder_decoder=True,device_map=device,torch_dtype=torch.float16)
add_special_tokens(model, processor.tokenizer)

In [ ]:
ds = Dataset.load_from_disk(data_path)
# ds = UICoderDataset(path=data_path,processor=processor,max_length=2048,max_patches=512,max_num=100,drop_longer=True,stage=1,preprocess=True, make_patches_while_training=True, workers=1)

In [ ]:
# image = Image.open('/data02/users/lz/code/image.jpg')
image = ds[216]['image']
print(image.size)
image

In [ ]:
model.eval()
with torch.no_grad():
    input = f'<!DOCTYPE html>'
    decoder_input_ids = processor.tokenizer.encode(input,return_tensors='pt',add_special_tokens=True)[...,:-1]
    encoding = processor(images=[image],text=[""],max_patches=1024,return_tensors='pt')
    item = {
        'decoder_input_ids': decoder_input_ids,
        'flattened_patches': encoding['flattened_patches'].half(),
        'attention_mask': encoding['attention_mask']
    }
    item = move_to_device(item,device)

    outputs = model.generate(**item,max_new_tokens=2048,eos_token_id=processor.tokenizer.eos_token_id,do_sample=True)
            
    prediction_html = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]


In [ ]:
prediction_html

In [ ]:
image